In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
stage_claims_dummy_file = dataiku.Dataset("stage_claims_dummy_file")
stage_claims_dummy_file_df = stage_claims_dummy_file.get_dataframe()

In [0]:
# Compute recipe outputs from inputs
# Compute recipe outputs from inputs
import random
from faker import Factory
import datetime

faker = Factory.create()

# get configs
create_or_append = dataiku.get_custom_variables()["create_or_append"]
total_claims = int(dataiku.get_custom_variables()["total_claims"])
append_claims = int(dataiku.get_custom_variables()["append_claims"])

#create static vars
claim_current_status = '01, 91, 02, , 01, 02, 01, 02'.split(', ')
provider_specialty = '1234, 1199, 2239, 6555, 1310'.split(',')
procedure_code = '9979, C4391, X123, D4578, 9N278, NSZKA, NYH4O, 43IXK, 3P29S, S3I46, 3K2US, 3272D, S3DSHJ, S2GJ3, 32WSS  ,    SN32K, VCGHD, , WSQQS, NULL,   23SDS, XSMJH   , 23KSWA'.split(', ')
diagnosis_code = 'SJ3YH, SG2IS, SO22S, SI2U3, G654R, LIG5UG, 796IO, 4JKY5, ABVCH, 65P3E, ,DIU3Y   , 43KJJ, 3SDB2   , 231YQ,   21SHJ, HSGZ21, 0, NULL, 2SAGH, S1MHQV, PJO7Y, 654GB, FD3R2, XENW2K,   43EDDS'.split(', ')
provider_id = '   WEFFER,    FKJ4BH3, FHU34KF   , NULL, F3K4HY   , NFKEWJAB, 23EDEW, 6476YHFG, KJH6K5J7Y    , 3KW45UTH, FKJSEBN, 4354FR, 343453VBDF, GH45K3, JGJKJBAT, KGTNRL, 754HTF, HFU8O34W , FE4U8O, LIU34H, '.split(', ')
provider_entity = 'P, P, P, P, P, P, P, I, G, F'.split(', ')

if(create_or_append == "create"):
    num_claims = total_claims
else:
    num_claims = append_claims

print("claims count: {0}".format(num_claims))

In [0]:
def date_between(d1, d2):
    f = '%b%d-%Y'
    return faker.date_time_between_dates(datetime.datetime.strptime(d1, f), datetime.datetime.strptime(d2, f))

# replaces data value with anomaly with defined probability, else return the original data
# Jamie refactor
def add_anomalies(value):
    # make 5 % chance of anomaly
    if random.randint(0, 100) <= 5:
        return value * random.randint(2,6)
    else:
        return value * random.randint(2,6)

def fakerecord_claims(num_claims):

    ## Define variables to generate 2 separate datasets for claims and claim line items
    claimrepeats = [faker.numerify('############') for _ in range(num_claims)]
    lineitemnumbers = [faker.random_int(min = 1, max = 20) for _ in range(num_claims)]
    claims = pd.DataFrame(np.repeat(claimrepeats, lineitemnumbers), columns = ['clcl_id'])
    claims['claim_lineitem_number'] = claims.groupby('clcl_id').cumcount()+1
    claims_li = []

    ## Claims
    unique_claims = pd.DataFrame(claims.clcl_id.unique(), columns = ['clcl_id'])
    for index, row in unique_claims.iterrows():
        claims_dict = {
            'clcl_id': unique_claims['clcl_id'][index],
            'claim_current_status': random.choice(claim_current_status),
            'provider_id': random.choice(provider_id),
            'clcl_input_dt': date_between('dec31-2013', 'dec31-2019'),
            'new_member_key': faker.random_int(min = 1, max = 67980)

        }
        claims_li.append(claims_dict)

    return claims_li

stage_claims_df = pd.DataFrame(fakerecord_claims(num_claims))

In [0]:
# Write recipe outputs
#changed by jonathan

In [0]:
#stage_claims_hdfs.write_with_schema(stage_claims_hdfs_df)



#added fs append
stage_claim_fs = dataiku.Dataset("stage_claims_fs")
stage_claim_fs.write_with_schema(stage_claims_df)